In [9]:
import pandas as pd
df = pd.read_csv(r'PEATONES_2020_mod.csv',encoding = "ISO-8859-1",delimiter=';',index_col=2)
df


,FECHA,HORA,PEATONES,NÚMERO_DISTRITO,DISTRITO,NOMBRE_VIAL,NÚMERO,CÓDIGO_POSTAL,OBSERVACIONES_DIRECCION,LATITUD,LONGITUD
IDENTIFICADOR,,,,,,,,,,,
PERM_PEA02_PM01,01/01/2020,0:00:00,497.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
PERM_PEA02_PM01,01/01/2020,0:15:00,244.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
PERM_PEA02_PM01,01/01/2020,0:30:00,179.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
PERM_PEA02_PM01,01/01/2020,0:45:00,174.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
PERM_PEA02_PM01,01/01/2020,1:00:00,131.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
...,...,...,...,...,...,...,...,...,...,...,...
PERM_PEA19_PM01,30/06/2020,22:45:00,565.0,1.0,Centro,Calle Bailén,6.0,28013.0,Acera Pares Datos en prueba,"40,417278","-3,713371"
PERM_PEA19_PM01,30/06/2020,23:00:00,415.0,1.0,Centro,Calle Bailén,6.0,28013.0,Acera Pares Datos en prueba,"40,417278","-3,713371"
PERM_PEA19_PM01,30/06/2020,23:15:00,551.0,1.0,Centro,Calle Bailén,6.0,28013.0,Acera Pares Datos en prueba,"40,417278","-3,713371"


In [10]:
# Normalizar nombres de columnas a minúsculas sin espacios para trabajar cómodo
df.columns = [c.strip().lower() for c in df.columns]

# Si pusiste index_col=2 antes y ahí venía distrito/otro, lo recuperamos a columna:
if df.index.name:
    df = df.reset_index()

df.head()
'''1) Identificar columnas clave (fecha, distrito, conteo)

El CSV de clase suele traer algo tipo fecha, distrito y peatones (o similar). Este helper las detecta aunque cambien un poco los nombres.'''

,IDENTIFICADOR,fecha,hora,peatones,número_distrito,distrito,nombre_vial,número,código_postal,observaciones_direccion,latitud,longitud
0,PERM_PEA02_PM01,01/01/2020,0:00:00,497.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
1,PERM_PEA02_PM01,01/01/2020,0:15:00,244.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
2,PERM_PEA02_PM01,01/01/2020,0:30:00,179.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
3,PERM_PEA02_PM01,01/01/2020,0:45:00,174.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"
4,PERM_PEA02_PM01,01/01/2020,1:00:00,131.0,1.0,Centro,Calle Fuencarral,22.0,28004.0,Calle peatonal Datos en prueba,"40,4220090","-3,7008917"


In [11]:
def find_col(df, keys):
    for c in df.columns:
        name = c.lower()
        if any(k in name for k in keys):
            return c
    raise KeyError(f"No encuentro columna con claves: {keys}")

col_fecha    = find_col(df, ['fech'])
col_distrito = find_col(df, ['distr', 'barrio', 'zona'])
col_conteo   = find_col(df, ['peat', 'conteo', 'trafic', 'person', 'cuenta'])

col_fecha, col_distrito, col_conteo


('fecha', 'número_distrito', 'peatones')

In [12]:
df[col_fecha] = pd.to_datetime(df[col_fecha], dayfirst=True, errors='coerce')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288755 entries, 0 to 288754
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   IDENTIFICADOR            288755 non-null  object        
 1   fecha                    288755 non-null  datetime64[ns]
 2   hora                     288755 non-null  object        
 3   peatones                 288752 non-null  float64       
 4   número_distrito          284872 non-null  float64       
 5   distrito                 284872 non-null  object        
 6   nombre_vial              284872 non-null  object        
 7   número                   284872 non-null  float64       
 8   código_postal            284872 non-null  float64       
 9   observaciones_direccion  284872 non-null  object        
 10  latitud                  284872 non-null  object        
 11  longitud                 284872 non-null  object        
dtypes: datetime64[ns

In [13]:
promedio_por_distrito = (
    df.groupby(col_distrito, as_index=True)[col_conteo]
      .mean()
      .sort_values(ascending=False)
      .to_frame('media_peatones')
)

promedio_por_distrito.head(10)


,media_peatones
número_distrito,
2.0,651.036304
1.0,488.248612
9.0,360.899778
7.0,269.302136
10.0,71.679353


In [14]:
stats_distrito_dia = (
    df.groupby([col_distrito, df[col_fecha].dt.date])[col_conteo]
      .agg(minimo='min', promedio='mean')
      .reset_index()
      .rename(columns={col_fecha: 'fecha'})
)

stats_distrito_dia.head(10)


,número_distrito,fecha,minimo,promedio
0,1.0,2020-01-01,10.0,416.541040
1,1.0,2020-01-02,6.0,457.966318
2,1.0,2020-01-03,7.0,459.554398
3,1.0,2020-01-04,5.0,439.265497
4,1.0,2020-01-05,4.0,628.626761
5,1.0,2020-01-06,3.0,505.361111
6,1.0,2020-01-07,5.0,437.080420
7,1.0,2020-01-08,6.0,389.370638
8,1.0,2020-01-09,4.0,363.725597
9,1.0,2020-01-10,9.0,372.548958


In [15]:
# Columna ajustada con lambda (+10%)
df['conteo_ajustado'] = df[col_conteo].apply(lambda x: x * 1.10)

# Total diario (sumando todos los distritos ese día) con el ajuste
total_diario_ajustado = (
    df.groupby(df[col_fecha].dt.date)['conteo_ajustado']
      .sum()
      .sort_values(ascending=False)
)

dia_max = total_diario_ajustado.idxmax()
valor_max = total_diario_ajustado.max()

dia_max, valor_max


(datetime.date(2020, 3, 8), 1227870.6)

In [16]:
top_distritos_dia_max = (
    df[df[col_fecha].dt.date == dia_max]
      .groupby(col_distrito)['conteo_ajustado']
      .sum()
      .sort_values(ascending=False)
)

top_distritos_dia_max.head(10)


número_distrito
1.0     911443.5
2.0     199684.1
7.0      65107.9
9.0      45202.3
10.0      6432.8
Name: conteo_ajustado, dtype: float64